<a href="https://colab.research.google.com/github/ujjwalgarg/learn_hugging_face/blob/main/Knowledge_Base_Q%26A_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch faiss-cpu sentence-transformers datasets pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
import transformers
import torch
import faiss
import sentence_transformers
import datasets

In [3]:
from datasets import load_dataset
dataset = load_dataset("squad_v2")  # closely resembles a company wiki
df = dataset["train"].to_pandas()[["context"]].drop_duplicates().head(50)  # Take 50 unique contexts

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [7]:
import pandas as pd

synthetic_data = [
  "Vacation policy: Employees get 15 days of paid leave annually.",
  "IT support: Contact helpdesk@company.com for technical issues.",
  "Dress code: Business casual is required Monday to Thursday.",
  "Expense Reimbursement: Travel expenses exceeding $500 require pre-approval from the department head.",
  "Internet Usage: Personal browsing is limited to non-peak hours and must not exceed 30 minutes per day.",
  "Social Media Policy: Employees must disclose their affiliation with the company when discussing company-related topics online.",
  "Confidentiality: All client information, including names and contact details, is strictly confidential and must not be shared outside the company.",
  "Performance Reviews: Annual performance reviews are based on a 360-degree feedback system, including peer and subordinate input.",
  "Gifts and Gratuities: Employees are prohibited from accepting gifts exceeding $50 in value from clients or vendors.",
  "Workplace Safety: All employees must complete annual safety training and adhere to posted safety protocols.",
  "Email Communication: Company email accounts are for business use only; personal emails are prohibited.",
  "Remote Work: Remote work is subject to manager approval and requires a dedicated workspace with secure internet access.",
  "Data Retention: All electronic data, including emails and documents, will be retained for seven years for compliance purposes.",
  "Conflict of Interest: Employees must disclose any potential conflicts of interest, including financial interests in competitor companies.",
  "Intellectual Property: All work created during employment is the property of the company.",
  "Whistleblower Policy: Employees can report suspected misconduct anonymously through the company hotline.",
  "Emergency Procedures: Employees must familiarize themselves with emergency evacuation plans and procedures.",
  "Substance Abuse: Drug and alcohol testing may be conducted randomly or for cause.",
  "Employee Monitoring: Company computers and networks may be monitored for security and compliance purposes.",
  "Outside Employment: Employees must obtain written approval for any outside employment that may conflict with their duties.",
  "Attendance: Employees must report absences at least one hour before their scheduled start time.",
  "Parking: Parking permits are assigned on a first-come, first-served basis.",
  "Health and Wellness: Employees are encouraged to participate in wellness programs offered by the company.",
  "Recruitment: Employees who refer successful candidates are eligible for a referral bonus.",
  "Termination: The company reserves the right to terminate employment with or without cause.",
  "Training and Development: Employees are required to complete mandatory training courses as assigned.",
  "Use of Company Vehicles: Company vehicles are for business use only and must be operated in accordance with traffic laws.",
  "Employee Grievance: Employees can file grievances with the HR department for resolution.",
  "Software Usage: Only company-approved software may be installed on company computers.",
  "Mobile Device Policy: Company-issued mobile devices must be used for business purposes only.",
  "Environmental Policy: Employees are expected to conserve energy and reduce waste.",
  "Customer Complaints: All customer complaints must be documented and forwarded to the customer service department.",
  "Non-Discrimination: The company prohibits discrimination based on race, color, religion, sex, national origin, age, disability, or genetic information.",
  "Flexible Work Arrangements: Flexible work arrangements, such as staggered hours, must be approved by the manager.",
  "Travel Policy: Employees traveling on company business must use approved travel agencies.",
  "Data Encryption: All sensitive data must be encrypted when stored or transmitted.",
  "Public Speaking: Employees must obtain approval before speaking on behalf of the company at public events.",
  "Internal Audits: The company conducts regular internal audits to ensure compliance with policies.",
  "Succession Planning: Key positions have designated successors to ensure business continuity.",
  "Document Retention: Physical documents must be stored in designated filing cabinets and shredded when no longer needed.",
  "Vendor Management: Vendor contracts must be reviewed and approved by the legal department.",
  "Business Continuity: The company has a business continuity plan to ensure operations during disruptions.",
  "Change Management: All changes to company policies must be documented and communicated to employees.",
  "Personal Electronics: Personal electronic devices, such as USB drives, are prohibited on company networks.",
  "Employee Discounts: Employees are eligible for discounts on company products or services.",
  "Dress Code Exceptions: Dress code exceptions may be granted for medical or religious reasons.",
  "Time Off Requests: Time off requests must be submitted at least two weeks in advance.",
  "Workplace Violence: The company has a zero-tolerance policy for workplace violence.",
  "Performance Improvement Plans: Employees with performance issues will be placed on a performance improvement plan.",
  "Employee Assistance Program: The company offers an employee assistance program for confidential counseling.",
  "Emergency Communication: Employees must provide up-to-date emergency contact information.",
  "Personal Calls: Personal calls should be limited to emergencies and brief conversations.",
  "Information Security: Employees must report any suspected security breaches immediately.",
  "Patent Disclosure: Any invention or discovery made during employment must be disclosed to the company.",
  "Public Records Requests: All public records requests must be handled by the legal department.",
  "Customer Data Privacy: Customer data must be handled in accordance with privacy laws and regulations.",
  "Software Licensing: Employees must adhere to software licensing agreements.",
  "Personal Data Usage: The company may collect and use personal data for legitimate business purposes.",
  "Overtime Policy: Overtime must be pre-approved by the manager and is paid at 1.5 times the regular rate.",
  "Workplace Ergonomics: Employees are encouraged to maintain ergonomic workstations.",
  "Social Events: Attendance at company social events is voluntary.",
  "Internal Communication: Employees must use approved communication channels for official business.",
  "Records Management: All business records must be maintained in accordance with company policies.",
  "Use of generative AI tools for work related tasks must be approved by the IT department.",
  "All recordings of meetings must be retained for 3 years, and are subject to legal discovery.",
  "Employees are required to report all software bugs, even minor ones, through the company's internal bug tracking system.",
  "Company provided laptops must have full disk encryption enabled at all times.",
  "Any employee using a VPN must use the company provided VPN.",
  "All data stored on cloud services must adhere to the company's data governance policy.",
  "Employees are required to complete annual cybersecurity training.",
  "Company provided smart phones must have a pin number and be set to auto lock after 2 minutes of inactivity.",
  "All external hard drives must be registered with the IT department.",
  "Employees are prohibited from installing any software that is not on the company approved list.",
  "Employees are required to report any suspected phishing emails to the IT department.",
  "Employees are required to obtain two factor authentication on all company accounts.",
  "Company provided devices are subject to random audits by the security department.",
  "All code written for the company is subject to code review.",
  "Employees are required to report any potential security threats.",
  "Employees are prohibited from sharing company passwords.",
  "All employees must complete a background check upon hire.",
  "Employees must disclose any criminal convictions.",
  "Employees are required to report any changes in their personal information.",
  "Employees are required to sign a non-disclosure agreement.",
  "Employees must report all injuries that occur on company property.",
  "Employees are required to wear company provided safety equipment when required.",
  "Employees are required to report any unsafe working conditions.",
  "Employees are prohibited from bringing firearms onto company property.",
  "Employees are required to report any suspected violations of company policy.",
  "Employees are required to report any conflicts of interest.",
  "Employees are required to report any gifts or gratuities received from clients or vendors.",
  "Employees are required to report any outside employment.",
  "Employees are required to report any political activity that may conflict with their duties.",
  "Employees are required to report any social media activity that may reflect negatively on the company.",
  "Employees are required to report any personal relationships with clients or vendors.",
  "Employees are required to report any financial interests in competitor companies.",
  "Employees are required to report any legal proceedings that may involve the company.",
  "Employees are required to report any government investigations that may involve the company.",
  "Employees are required to report any media inquiries that may involve the company."
]
synthetic_data_df = pd.DataFrame(synthetic_data, columns=["context"])
df = pd.concat([df, synthetic_data_df], ignore_index=True)

In [8]:
df

,context
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,Following the disbandment of Destiny's Child i...
2,"A self-described ""modern-day feminist"", Beyonc..."
3,"Beyoncé Giselle Knowles was born in Houston, T..."
4,Beyoncé attended St. Mary's Elementary School ...
...,...
143,Employees are required to report any personal ...
144,Employees are required to report any financial...
145,Employees are required to report any legal pro...
146,Employees are required to report any governmen...


In [9]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
knowledge_base = df.copy()
knowledge_base["text"] = knowledge_base["context"]

In [12]:
embeddings = embedder.encode(knowledge_base["text"].tolist(), convert_to_numpy=True)
print(embeddings.shape)  # [num_entries, 384]

(148, 384)


In [13]:
import faiss
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2").to("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [19]:
def rag_chatbot(query, top_k=3):
    # Encode query
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    contexts = [knowledge_base["text"].iloc[idx] for idx in indices[0]]

    # Build prompt
    prompt = f"Query: {query}\nContext: {' '.join(contexts)}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()

In [21]:
queries = [
    "What’s the vacation policy?",
    "How do I contact IT support?",
    "What’s the dress code?",
    "What's the smoking policy?",
    "When do I get paid for my work?",
    "Do I have to take permission everytime I have to pee?"
]
for q in queries:
    print(f"Q: {q}")
    print(f"A: {rag_chatbot(q)}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What’s the vacation policy?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A: This is a statement made by the Department of Transportation and the National Weather Service. This statement is only available on the website of the Secretary of State and only on website at www.nw.gov.

Q: How do I contact IT support?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

A: The dress codes, which are not required, are designed to be worn by employees in accordance with the Dress Codes. All dress Code exceptions are subject to the following guidelines:
The dress CODE must be clearly stated in the name of the company or that the employee must wear a suit, dress, or dress. The uniform code must not be used in a manner that violates the Code.

Q: What's the smoking policy?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A: No. When you own a mobile device, you are required to carry a firearm or to be licensed to use it. No firearm is prohibited in an emergency situation. You must have a valid business license and have an annual permit to purchase a rifle.

Q: When do I get paid for my work?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A: Employers should have a list of hourly and weekly hours they have worked for them. For instance, if you have more than one week to work for one hour, you may be required to return one to one. The employer will not be able to provide a full time, full-time job. Employees who are not required by their employer to take an unpaid leave will be subject to the overtime

Q: Do I have to take permission everytime I have to pee?
A: It is the right thing to do.



In [ ]:
# further enhancements
# fine tune
# evaluation criteria
# hyperparameter tuning